<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/Speech_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Setting Up the Environment in Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
# Install required libraries
!pip install opensmile speechbrain librosa

In [4]:
# Extract .tgz files
import tarfile
import os

data_path = '/content/drive/MyDrive/Voice/'  # Adjust to your dataset path
tgz_files = [
    'ADReSSo21-progression-train.tgz',
    'ADReSSo21-progression-test.tgz',
    'ADReSSo21-diagnosis-train.tgz'
]

for tgz in tgz_files:
    tar = tarfile.open(os.path.join(data_path, tgz), 'r:gz')
    tar.extractall('/content/data')
    tar.close()

# Step 2: Loading and Preprocessing Datasets

## Load Label Files

In [5]:
import pandas as pd

# Load label files
task1 = pd.read_csv(os.path.join(data_path, 'task1.csv'))  # AD classification labels
task2 = pd.read_csv(os.path.join(data_path, 'task2.csv'))  # MMSE regression labels
task3 = pd.read_csv(os.path.join(data_path, 'task3.csv'))  # Cognitive decline labels

In [6]:
task1.head()

,ID,Dx
0,adrsdt15,Control
1,adrsdt40,Control
2,adrsdt26,Control
3,adrsdt67,Control
4,adrsdt58,Control


- task1.csv: ID and AD/Control labels for diagnosis (AD classification).
- task2.csv: ID and MMSE scores for regression.
- task3.csv: ID and Decline/Non-Decline labels for cognitive decline prediction.

## Map Audio Files to Labels

In [7]:
# Function to map audio files to labels
def load_audio_label_pairs(audio_dir, label_df, label_col, id_col='ID'):
    audio_files = []
    labels = []
    for _, row in label_df.iterrows():
        audio_path = os.path.join(audio_dir, f"{row[id_col]}.wav")
        if os.path.exists(audio_path):
            audio_files.append(audio_path)
            labels.append(row[label_col])
    return audio_files, labels

# AD Classification (Cookie Theft, AD vs CN)
ad_audio_dir = '/content/data/ADReSSo21/diagnosis/train/audio'
ad_audio_files, ad_labels = load_audio_label_pairs(
    ad_audio_dir, task1, 'Dx'
)

# MMSE Regression (same audio files as AD classification)
mmse_audio_files, mmse_labels = load_audio_label_pairs(
    ad_audio_dir, task2, 'MMSE'
)

# Cognitive Decline (Category Fluency Task)
prog_audio_dir = '/content/data/ADReSSo21/progression/train/audio'
decline_audio_files, decline_labels = load_audio_label_pairs(
    prog_audio_dir, task3, 'Decline'
)

- For AD classification, use ADReSSo21-diagnosis-train.tgz audio files and task1.csv.
- For MMSE regression, use the same audio files as AD classification, paired with task2.csv.
- For cognitive decline, use ADReSSo21-progression-train.tgz audio files and task3.csv
- Ensure audio files match the IDs in the CSV files. If some IDs are missing audio, you may need to filter them out.
- The segmentation directories contain CSV files with transcriptions or timings, which can be used for linguistic feature extraction.

# Step 3: Feature Extraction

You need to extract acoustic and linguistic features as specified.

Acoustic Features
- eGeMAPS: Use opensmile to extract eGeMAPS features.
- Active Data Representation (ADR): This may require a custom implementation or pre-trained model.